In [72]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from ImageDisplayer import show_img_custom
import webcolors

In [5]:
brown_image_path = './test_images/brown.png'
img = cv2.imread(brown_image_path)

test = np.array(list(img))

In [17]:

n = test.shape[0]
m = test.shape[1]

cnt = {}
for i in range(n):
    for j in range(m):
        cur = (test[i][j][0], test[i][j][1], test[i][j][2])
        if cur not in cnt:
            cnt[cur] = 0
        cnt[cur] += 1
    
sorted = []    
for i in cnt.keys():
    sorted.append((cnt[i], i))

In [21]:
sorted.sort()
print(sorted[0][1], sorted[1][1])

(37, 50, 56) (39, 50, 55)


In [58]:
def find_main_objects_grayscale(image_path, area_threshold):
    
    image = cv2.imread(image_path)
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    _, binary_image = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    main_object_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > area_threshold]

    result_image = image.copy()
    cv2.drawContours(result_image, main_object_contours, -1, (0, 255, 0), 2)

    # Display the result image
    show_img_custom(result_image, 'Main Objects (Grayscale)')

    return main_object_contours


In [59]:
# Example usage:
# run_way_path = './test_images/runway_50.png'
# maryland_path = './test_images/maryland_test_50.png'
brown_image_path = './test_images/brown.png'

brown_img = cv2.imread(brown_image_path)
# maryland_img = cv2.imread(maryland_path)
# runway_img = cv2.imread(run_way_path)

n = np.array(brown_img).shape[0]
m = np.array(brown_img).shape[1]

area_threshold = n * m * 0.01 # Adjust this threshold according to your requirements

main_objects_grayscale = find_main_objects_grayscale(brown_image_path, area_threshold)
print(f"Number of main objects found: {len(main_objects_grayscale)}")

Number of main objects found: 1


In [65]:
import cv2
import numpy as np

def get_contour_values_hsv(image_path, area_threshold=.001):
    # Read the image
    image = cv2.imread(image_path)

    # Convert the image to HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define a lower and upper threshold for the color range you are interested in
    lower_threshold = np.array([0, 0, 0])  # Example: Black
    upper_threshold = np.array([179, 255, 50])  # Example: Yellow

    # Create a mask using the inRange function
    mask = cv2.inRange(hsv, lower_threshold, upper_threshold)

    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filter contours based on area threshold
    main_object_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > area_threshold]

    return main_object_contours

In [66]:
def find_common_colors_hsv(image, contours, num_colors=2):
    # Create a mask to extract the region of interest (ROI) within each contour
    mask = np.zeros_like(image)
    cv2.drawContours(mask, contours, -1, (255, 255, 255), thickness=cv2.FILLED)

    # Convert the image to HSV
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Iterate over each contour and extract the color information
    colors_hsv = []
    for contour in contours:
        # Extract the ROI using the mask
        roi = cv2.bitwise_and(hsv_image, mask)
        
        # Mask the image to keep only the pixels within the contour
        contour_mask = np.zeros_like(hsv_image)
        cv2.drawContours(contour_mask, [contour], -1, (255, 255, 255), thickness=cv2.FILLED)
        roi = cv2.bitwise_and(roi, contour_mask)

        # Calculate the average color within the contour
        average_color_hsv = np.mean(roi, axis=(0, 1))
        colors_hsv.append(average_color_hsv)

    # Find the two most common colors in HSV format
    unique_colors_hsv, counts = np.unique(colors_hsv, axis=0, return_counts=True)
    sorted_indices = np.argsort(-counts)
    most_common_colors_hsv = unique_colors_hsv[sorted_indices[:num_colors]]

    return most_common_colors_hsv

In [67]:
image_path = brown_image_path
area_threshold = .001  # Adjust this threshold according to your requirements

contour_values_hsv = get_contour_values_hsv(image_path, area_threshold)

find_common_colors_hsv(brown_img, contour_values_hsv, 2)

array([], dtype=float64)

In [77]:
def rgb_to_hex(rgb):
    return "#{:02x}{:02x}{:02x}".format(rgb[0], rgb[1], rgb[2])

def hex_to_color_name(hex_code):
    try:
        color_name = webcolors.hex_to_name(hex_code)
        return color_name
    except ValueError:
        return "Unknown"

def identify_major_colors(image_path, top_n=2):
    # Load the image using Pillow
    image = Image.open(image_path)

    # Convert the image to a NumPy array
    image_np = np.array(image)

    # Convert the image to RGB (if it's in a different mode)
    image_rgb = cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB)

    # Define a mask to exclude the background
    # This mask assumes that the background is a solid color (you may need to adjust it)
    background_mask = (image_rgb[:, :, 0] != 255) | (image_rgb[:, :, 1] != 255) | (image_rgb[:, :, 2] != 255)

    # Apply the mask to the image
    image_rgb_no_bg = image_rgb[background_mask]

    # Identify unique colors in the sign and symbol (excluding background)
    unique_colors, counts = np.unique(image_rgb_no_bg, axis=0, return_counts=True)

    # Sort colors based on their counts (areas)
    sorted_indices = np.argsort(-counts)
    sorted_colors = unique_colors[sorted_indices]

    # Take the top N colors
    top_colors = sorted_colors[:top_n]

    return top_colors

# Replace 'your_image_path.jpg' with the path to your image file
image_path = './test_images/colorer.png'
result = identify_major_colors(image_path)

print("Major Colors:")
for color in result:
    hex_color = rgb_to_hex(color)
    color_name = hex_to_color_name(hex_color)
    print(f"Hex Code: {hex_color}, Color Name: {color_name}")


Major Colors:
Hex Code: #7bb8e5, Color Name: Unknown
